### IMPORTS

In [9]:
%pip install azure-identity==1.13.0b4

  Using cached azure_identity-1.13.0b4-py3-none-any.whl.metadata (65 kB)
  Using cached azure_core-1.34.0-py3-none-any.whl.metadata (42 kB)
  Using cached msal-1.32.3-py3-none-any.whl.metadata (11 kB)
  Using cached msal_extensions-1.3.1-py3-none-any.whl.metadata (7.8 kB)
Using cached azure_identity-1.13.0b4-py3-none-any.whl (144 kB)
Using cached azure_core-1.34.0-py3-none-any.whl (207 kB)
Using cached msal-1.32.3-py3-none-any.whl (115 kB)
Using cached msal_extensions-1.3.1-py3-none-any.whl (20 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
%pip install azure-search-documents==11.4.0b3

  Using cached azure_search_documents-11.4.0b3-py3-none-any.whl.metadata (18 kB)
  Using cached azure_common-1.1.28-py2.py3-none-any.whl.metadata (5.0 kB)
  Using cached isodate-0.7.2-py3-none-any.whl.metadata (11 kB)
Using cached azure_search_documents-11.4.0b3-py3-none-any.whl (296 kB)
Using cached azure_common-1.1.28-py2.py3-none-any.whl (14 kB)
Using cached isodate-0.7.2-py3-none-any.whl (22 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
%pip install azure-storage-blob==12.14.1

  Using cached azure_storage_blob-12.14.1-py3-none-any.whl.metadata (26 kB)
Using cached azure_storage_blob-12.14.1-py3-none-any.whl (383 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
%pip install azure-ai-formrecognizer==3.2.1

  Using cached azure_ai_formrecognizer-3.2.1-py3-none-any.whl.metadata (57 kB)
  Using cached msrest-0.7.1-py3-none-any.whl.metadata (21 kB)
  Using cached requests_oauthlib-2.0.0-py2.py3-none-any.whl.metadata (11 kB)
  Using cached oauthlib-3.2.2-py3-none-any.whl.metadata (7.5 kB)
Using cached azure_ai_formrecognizer-3.2.1-py3-none-any.whl (228 kB)
Using cached msrest-0.7.1-py3-none-any.whl (85 kB)
Using cached requests_oauthlib-2.0.0-py2.py3-none-any.whl (24 kB)
Using cached oauthlib-3.2.2-py3-none-any.whl (151 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
%pip install pypdf




  Using cached pypdf-5.6.0-py3-none-any.whl.metadata (7.2 kB)
Using cached pypdf-5.6.0-py3-none-any.whl (304 kB)



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import argparse
import glob
import html
import io
import re
import time
from pypdf import PdfReader, PdfWriter
from azure.identity import AzureDeveloperCliCredential
from azure.core.credentials import AzureKeyCredential
from azure.storage.blob import BlobServiceClient
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import *
from azure.search.documents import SearchClient
from azure.ai.formrecognizer import DocumentAnalysisClient

In [6]:
from config import *

In [7]:
search_creds = AzureKeyCredential(searchkey)
storage_creds = storagekey

In [8]:
MAX_SECTION_LENGTH = 1000
SENTENCE_SEARCH_LIMIT = 100
SECTION_OVERLAP = 100

### FUNCTIONS

In [9]:
def blob_name_from_file_page(filename, page = 0):
    if os.path.splitext(filename)[1].lower() == ".pdf":
        return os.path.splitext(os.path.basename(filename))[0] + f"-{page}" + ".pdf"
    else:
        return os.path.basename(filename)

In [10]:
def upload_blobs(filename):
    blob_service = BlobServiceClient(account_url=f"https://{storageaccount}.blob.core.windows.net", credential=storage_creds)
    blob_container = blob_service.get_container_client(container)
    if not blob_container.exists():
        blob_container.create_container()

    # if file is PDF split into pages and upload each page as a separate blob
    if os.path.splitext(filename)[1].lower() == ".pdf":
        reader = PdfReader(filename)
        pages = reader.pages
        for i in range(len(pages)):
            blob_name = blob_name_from_file_page(filename, i)
            if verbose: print(f"\tUploading blob for page {i} -> {blob_name}")
            f = io.BytesIO()
            writer = PdfWriter()
            writer.add_page(pages[i])
            writer.write(f)
            f.seek(0)
            blob_container.upload_blob(blob_name, f, overwrite=True)
    else:
        blob_name = blob_name_from_file_page(filename)
        with open(filename,"rb") as data:
            blob_container.upload_blob(blob_name, data, overwrite=True)


def table_to_html(table):
    table_html = "<table>"
    rows = [sorted([cell for cell in table.cells if cell.row_index == i], key=lambda cell: cell.column_index) for i in range(table.row_count)]
    for row_cells in rows:
        table_html += "<tr>"
        for cell in row_cells:
            tag = "th" if (cell.kind == "columnHeader" or cell.kind == "rowHeader") else "td"
            cell_spans = ""
            if cell.column_span > 1: cell_spans += f" colSpan={cell.column_span}"
            if cell.row_span > 1: cell_spans += f" rowSpan={cell.row_span}"
            table_html += f"<{tag}{cell_spans}>{html.escape(cell.content)}</{tag}>"
        table_html +="</tr>"
    table_html += "</table>"
    return table_html

def get_document_text(filename):
    offset = 0
    page_map = []
    if localpdfparser:
        reader = PdfReader(filename)
        pages = reader.pages
        for page_num, p in enumerate(pages):
            page_text = p.extract_text()
            page_map.append((page_num, offset, page_text))
            offset += len(page_text)
    else:
        if verbose: print(f"Extracting text from '{filename}' using Azure Form Recognizer")
        form_recognizer_client = DocumentAnalysisClient(endpoint=f"https://{formrecognizerservice}.cognitiveservices.azure.com/", credential=formrecognizer_creds, headers={"x-ms-useragent": "azure-search-chat-demo/1.0.0"})
        with open(filename, "rb") as f:
            poller = form_recognizer_client.begin_analyze_document("prebuilt-layout", document = f)
        form_recognizer_results = poller.result()

        for page_num, page in enumerate(form_recognizer_results.pages):
            tables_on_page = [table for table in form_recognizer_results.tables if table.bounding_regions[0].page_number == page_num + 1]

            # mark all positions of the table spans in the page
            page_offset = page.spans[0].offset
            page_length = page.spans[0].length
            table_chars = [-1]*page_length
            for table_id, table in enumerate(tables_on_page):
                for span in table.spans:
                    # replace all table spans with "table_id" in table_chars array
                    for i in range(span.length):
                        idx = span.offset - page_offset + i
                        if idx >=0 and idx < page_length:
                            table_chars[idx] = table_id

            # build page text by replacing charcters in table spans with table html
            page_text = ""
            added_tables = set()
            for idx, table_id in enumerate(table_chars):
                if table_id == -1:
                    page_text += form_recognizer_results.content[page_offset + idx]
                elif not table_id in added_tables:
                    page_text += table_to_html(tables_on_page[table_id])
                    added_tables.add(table_id)

            page_text += " "
            page_map.append((page_num, offset, page_text))
            offset += len(page_text)

    return page_map

In [13]:
def split_text(page_map):
    SENTENCE_ENDINGS = [".", "!", "?"]
    WORDS_BREAKS = [",", ";", ":", " ", "(", ")", "[", "]", "{", "}", "\t", "\n"]
    if verbose: print(f"Splitting '{filename}' into sections")

    def find_page(offset):
        l = len(page_map)
        for i in range(l - 1):
            if offset >= page_map[i][1] and offset < page_map[i + 1][1]:
                return i
        return l - 1

    all_text = "".join(p[2] for p in page_map)
    length = len(all_text)
    start = 0
    end = length
    while start + SECTION_OVERLAP < length:
        last_word = -1
        end = start + MAX_SECTION_LENGTH

        if end > length:
            end = length
        else:
            # Try to find the end of the sentence
            while end < length and (end - start - MAX_SECTION_LENGTH) < SENTENCE_SEARCH_LIMIT and all_text[end] not in SENTENCE_ENDINGS:
                if all_text[end] in WORDS_BREAKS:
                    last_word = end
                end += 1
            if end < length and all_text[end] not in SENTENCE_ENDINGS and last_word > 0:
                end = last_word # Fall back to at least keeping a whole word
        if end < length:
            end += 1

        # Try to find the start of the sentence or at least a whole word boundary
        last_word = -1
        while start > 0 and start > end - MAX_SECTION_LENGTH - 2 * SENTENCE_SEARCH_LIMIT and all_text[start] not in SENTENCE_ENDINGS:
            if all_text[start] in WORDS_BREAKS:
                last_word = start
            start -= 1
        if all_text[start] not in SENTENCE_ENDINGS and last_word > 0:
            start = last_word
        if start > 0:
            start += 1

        section_text = all_text[start:end]
        yield (section_text, find_page(start))

        last_table_start = section_text.rfind("<table")
        if (last_table_start > 2 * SENTENCE_SEARCH_LIMIT and last_table_start > section_text.rfind("</table")):
            # If the section ends with an unclosed table, we need to start the next section with the table.
            # If table starts inside SENTENCE_SEARCH_LIMIT, we ignore it, as that will cause an infinite loop for tables longer than MAX_SECTION_LENGTH
            # If last table starts inside SECTION_OVERLAP, keep overlapping
            if verbose: print(f"Section ends with unclosed table, starting next section with the table at page {find_page(start)} offset {start} table start {last_table_start}")
            start = min(end - SECTION_OVERLAP, start + last_table_start)
        else:
            start = end - SECTION_OVERLAP
        
    if start + SECTION_OVERLAP < end:
        yield (all_text[start:end], find_page(start))

def create_sections(filename, page_map):
    for i, (section, pagenum) in enumerate(split_text(page_map)):
        yield {
            "id": re.sub("[^0-9a-zA-Z_-]","_",f"{filename}-{i}"),
            "content": section,
            "category": category,
            "sourcepage": blob_name_from_file_page(filename, pagenum),
            "sourcefile": filename
        }

def create_search_index():
    if verbose: print(f"Ensuring search index {index} exists")
    index_client = SearchIndexClient(endpoint=f"https://knavitpolicychatbot1.search.windows.net",
                                     credential=search_creds)
    if index not in index_client.list_index_names():
        search_index = SearchIndex(
            name=index,
            fields=[
                SimpleField(name="id", type="Edm.String", key=True),
                SearchableField(name="content", type="Edm.String", analyzer_name="en.microsoft"),
                SimpleField(name="category", type="Edm.String", filterable=True, facetable=True),
                SimpleField(name="sourcepage", type="Edm.String", filterable=True, facetable=True),
                SimpleField(name="sourcefile", type="Edm.String", filterable=True, facetable=True)
            ],
            semantic_settings=SemanticSettings(
                configurations=[SemanticConfiguration(
                    name='default',
                    prioritized_fields=PrioritizedFields(
                        title_field=None, prioritized_content_fields=[SemanticField(field_name='content')]))])
        )
        if verbose: print(f"Creating {index} search index")
        index_client.create_index(search_index)
    else:
        if verbose: print(f"Search index {index} already exists")

def index_sections(filename, sections):
    if verbose: print(f"Indexing sections from '{filename}' into search index '{index}'")
    search_client = SearchClient(endpoint=f"https://knavitpolicychatbot1.search.windows.net",
                                    index_name=index,
                                    credential=search_creds)
    i = 0
    batch = []
    for s in sections:
        batch.append(s)
        i += 1
        if i % 1000 == 0:
            results = search_client.upload_documents(documents=batch)
            succeeded = sum([1 for r in results if r.succeeded])
            if verbose: print(f"\tIndexed {len(results)} sections, {succeeded} succeeded")
            batch = []

    if len(batch) > 0:
        results = search_client.upload_documents(documents=batch)
        succeeded = sum([1 for r in results if r.succeeded])
        if verbose: print(f"\tIndexed {len(results)} sections, {succeeded} succeeded")

def remove_from_index(filename):
    if verbose: print(f"Removing sections from '{filename or '<all>'}' from search index '{index}'")
    search_client = SearchClient(endpoint=f"",
                                    index_name=index,
                                    credential=search_creds)
    while True:
        filter = None if filename == None else f"sourcefile eq '{os.path.basename(filename)}'"
        r = search_client.search("", filter=filter, top=1000, include_total_count=True)
        if r.get_count() == 0:
            break
        r = search_client.delete_documents(documents=[{ "id": d["id"] } for d in r])
        if verbose: print(f"\tRemoved {len(r)} sections from index")
        # It can take a few seconds for search results to reflect changes, so wait a bit
        time.sleep(2)



In [14]:
create_search_index()

Ensuring search index knavitpolicychatbot1 exists
Search index knavitpolicychatbot1 already exists


In [15]:
FILE_PATH

'C:\\Users\\Rupam Patil\\Downloads\\AZURE-AI-SEARCH-main\\docs'

In [16]:
for filename in glob.glob(FILE_PATH + "/*.pdf"):
    if verbose: print(f"Processing '{filename}'")
    upload_blobs(filename)
    page_map = get_document_text(filename)
    sections = create_sections(filename, page_map)
    index_sections(filename, sections)

Processing 'C:\Users\Rupam Patil\Downloads\AZURE-AI-SEARCH-main\docs\Information-Security-Policy.pdf'


Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 42 0 (offset 0)
Ignoring wrong pointing object 45 0 (offset 0)
Ignoring wrong pointing object 48 0 (offset 0)
Ignoring wrong pointing object 51 0 (offset 0)
Ignoring wrong pointing object 71 0 (offset 0)


	Uploading blob for page 0 -> Information-Security-Policy-0.pdf
	Uploading blob for page 1 -> Information-Security-Policy-1.pdf
	Uploading blob for page 2 -> Information-Security-Policy-2.pdf
	Uploading blob for page 3 -> Information-Security-Policy-3.pdf
	Uploading blob for page 4 -> Information-Security-Policy-4.pdf
	Uploading blob for page 5 -> Information-Security-Policy-5.pdf
	Uploading blob for page 6 -> Information-Security-Policy-6.pdf
	Uploading blob for page 7 -> Information-Security-Policy-7.pdf
	Uploading blob for page 8 -> Information-Security-Policy-8.pdf
	Uploading blob for page 9 -> Information-Security-Policy-9.pdf
	Uploading blob for page 10 -> Information-Security-Policy-10.pdf
	Uploading blob for page 11 -> Information-Security-Policy-11.pdf
	Uploading blob for page 12 -> Information-Security-Policy-12.pdf
	Uploading blob for page 13 -> Information-Security-Policy-13.pdf
	Uploading blob for page 14 -> Information-Security-Policy-14.pdf
	Uploading blob for page 15 ->

Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 42 0 (offset 0)
Ignoring wrong pointing object 45 0 (offset 0)
Ignoring wrong pointing object 48 0 (offset 0)
Ignoring wrong pointing object 51 0 (offset 0)
Ignoring wrong pointing object 71 0 (offset 0)


Indexing sections from 'C:\Users\Rupam Patil\Downloads\AZURE-AI-SEARCH-main\docs\Information-Security-Policy.pdf' into search index 'knavitpolicychatbot1'
Splitting 'C:\Users\Rupam Patil\Downloads\AZURE-AI-SEARCH-main\docs\Information-Security-Policy.pdf' into sections


HttpResponseError: () The request is invalid. Details: The property 'id' does not exist on type 'search.documentFields' or is not present in the API version '2021-04-30-Preview'. Make sure to only use property names that are defined by the type.
Code: 
Message: The request is invalid. Details: The property 'id' does not exist on type 'search.documentFields' or is not present in the API version '2021-04-30-Preview'. Make sure to only use property names that are defined by the type.